In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np

from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Conv1D, GRU, Bidirectional, Dropout
from keras.utils import plot_model

import matplotlib.pyplot as plt
import time
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import datetime
import math
import warnings
warnings.filterwarnings("ignore")
import glob
from sklearn.metrics import mean_squared_error, mean_absolute_error

# **Functions**

### **Provide path for raw REDD data**

In [ ]:
def read_label():
    label = {}
    for i in range(1,4):
        hi = '/content/gdrive/MyDrive/energy_disaggregation/windowGRU/data/house_{}/labels.dat'.format(i)
        label[i] = {}
        with open(hi) as f:
            for line in f:
                splitted_line = line.split(' ')
                label[i][int(splitted_line[0])] = splitted_line[1].strip() + '_' + splitted_line[0]
    return label


#-------------------------------------------------------------------------------------------------------------------

def read_merge_data(house):
    path = '/content/gdrive/MyDrive/energy_disaggregation/windowGRU/data/house_{}/'.format(house)
    
    
    file = path + 'channel_1.dat'

    df = pd.read_table(file, sep = ' ', names = ['unix_time', labels[house][1]], 
                                       dtype = {'unix_time': 'int64', labels[house][1]:'float64'}) 

    num_apps = len(glob.glob(path + 'channel*'))

    
    for i in range(2, num_apps + 1):
        file = path + 'channel_{}.dat'.format(i)
        data = pd.read_table(file, sep = ' ', names = ['unix_time', labels[house][i]], 
                                       dtype = {'unix_time': 'int64', labels[house][i]:'float64'})
        df = pd.merge(df, data, how = 'inner', on = 'unix_time')
    df['timestamp'] = df['unix_time'].astype("datetime64[s]")
    df = df.set_index(df['timestamp'].values)
    df.drop(['unix_time','timestamp'], axis=1, inplace=True)
    return df

#-------------------------------------------------------------------------------------------------------------------

def uniform_label(df):
  if 'microwave_11' in df.columns:   
    df = df.rename(columns={'microwave_11': 'microwave'})

  if 'microwave_6' in df.columns:
    df = df.rename(columns={'microwave_6': 'microwave'})

  if 'microwave_16' in df.columns:
    df = df.rename(columns={'microwave_16': 'microwave'})

  return df

#----------------------------------------------------------------------------------------------------------------------

def add_houses(df_list, appliance):
  merged_df = pd.DataFrame()
  for df in df_list:
    df = uniform_label(df)

    df["aggregate_mains"] = df['mains_1'] + df['mains_2']
    df = df[['aggregate_mains',appliance]]
    
    merged_df = pd.concat([merged_df,df])
  return merged_df

#-----------------------------------------------------------------------------------------------------------------------

def max_value(df):
  return max(df['aggregate_mains'])


#-----------------------------------------------------------------------------------------------------------------------


def preprocess_data(df, appliance, nmax):

  df['aggregate_mains'] = df['aggregate_mains']/ nmax

  df[appliance] = df[appliance]/ nmax
  
  return df

#----------------------------------------------------------------------------------------------------------------------
def create_window_chunk(df, sequence_length):
  result = []
  for index in range(len(df) - sequence_length):
      result.append(df[index: index + sequence_length])

  return result

#----------------------------------------------------------------------------------------------------------------------

def _create_model(window_size):
    '''Creates the GRU architecture described in the paper
    '''
    model = Sequential()

    # 1D Conv
    model.add(Conv1D(16, 4, activation='relu', input_shape=(window_size,1), padding="same", strides=1))

    #Bi-directional GRUs
    model.add(Bidirectional(GRU(64, activation='relu', return_sequences=True), merge_mode='concat'))
    model.add(Dropout(0.5))
    model.add(Bidirectional(GRU(128, activation='relu', return_sequences=False), merge_mode='concat'))
    model.add(Dropout(0.5))

    # Fully Connected Layers
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='linear'))

    model.compile(loss='mse', optimizer='adam')
    print(model.summary())

    return model

#---------------------------------------------------------------------------------------------------------------------


###*Provide path of saved model*

In [ ]:
# saved_model = "gdrive/MyDrive/saved_model/new_microwave_train_2_3_test_1_e_20_b_128_s_t_v_0.33_w_10_model.h5"
saved_model = ""

### *max value for house 2 & 3*

In [ ]:
nmax = 8059.92

### *Initializing the window size and dataframe dictionary*



In [ ]:
window_size = 50
df_dic={}

#**Loading saved model**

In [ ]:
model = _create_model(window_size)


model = load_model(saved_model)

### *Preparing and preprocessing test data - house 1*

In [ ]:
labels = read_label()
for i in range(1,4):
    df_dic[i] = read_merge_data(i)

In [ ]:
df_list = [df_dic[1]]
dfs = add_houses(df_list, 'dishwasher')
df_test = preprocess_data(dfs, "dishwasher", nmax)

In [ ]:
sequence_length = window_size

x_test = create_window_chunk(df_test['aggregate_mains'], sequence_length)

y_test = df_test['dishwasher'][sequence_length:]

In [ ]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [ ]:
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

###*Predicting on the test*

In [ ]:
pred_test = model.predict(x_test, batch_size=512)

In [ ]:
pred_test = np.reshape(pred_test, (len(pred_test)))

### *Denormalized the test and predicted data*

In [ ]:
y_test_d = y_test * nmax
y_pred_d = pred_test* nmax

###*Removing the predicted readings which are below average dishwasher reading , observed dishwasher reading as 800*

In [ ]:
y_pred_d = [0 if y<800 else y for y in list(y_pred_d)]

#**Data Visualization**

In [ ]:
plt.figure(figsize = (30, 8))
plt.plot(y_test_d, color='pink', label="ground dishwaser")
plt.plot(y_pred_d, color='red', label="predicted dishwaser")
plt.show

In [ ]:
plt.figure(figsize = (30, 8))
plt.plot(y_test_d[2000:3000], color='pink', label="ground dishwaser")
plt.plot(y_pred_d[2000:3000], color='red', label="predicted dishwaser")

plt.show

#**Evaluation Metrics**

In [ ]:
# Calculating MAE on test data
mean_absolute_error(y_test_d, y_pred_d)

In [ ]:
# Calculating RMSE on test data
math.sqrt(mean_squared_error(y_test_d, y_pred_d))